## MA-GRYP Task

Labeling the candlestick combination of R,G,Y and Price from top to bottom

PRGY refering to Price(Close Price), MovingAverage200(R), MovingAverage100(G), MovingAverage14(Y)

Ratio between these 4 lines also labeled in %

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')
# from zigzag import *
from tqdm import tqdm

In [2]:

data = pd.read_csv(f'C:\\Users\\HFY\\Desktop\\capstone\\data\\Data_OANDA\\GBP_USD_M15_2005_202206.csv', encoding='utf-8',header=0)
#data['datetime'] = pd.to_datetime(data['datetime'], format='%Y-%m-%dT%H:%M:%S.000000000Z')
data.reset_index(drop=True, inplace=True)

In [ ]:
# done: "AUD_USD_M15_2005_202206", 'EUR_USD_M15_2005_202206',

FILE_NAME_LIST = [
'GBP_USD_M15_2005_202206',
'USD_CAD_M15_2005_202206','USD_JPY_M15_2005_202206','USD_CHF_M15_2005_202206',
'NZD_USD_M15_2005_202206','AUD_CAD_M15_2005_202206'
]

#change directroy
data_ls = []
for i in FILE_NAME_LIST:
    file = pd.read_csv(f'C:\\Users\\HFY\\Desktop\\capstone\\data\\Data_OANDA\\{i}.csv', encoding='utf-8',header=0)
    file.reset_index(drop=True, inplace=True)
    file['datetime'] = pd.to_datetime(data['datetime'], format='%Y-%m-%dT%H:%M:%S.000000000Z')
    data_ls.append(file)

In [ ]:
data  = data[0:1000]
# moving average 200
MA200 = data['close'].rolling(window =200).mean()
# moving average 100
MA100 = data['close'].rolling(window =100).mean()
# moving average 14
MA14 = data['close'].rolling(window =14).mean()

fig = go.Figure(data=[go.Candlestick(x=data.index,
                open=data['open'],
                high=data['high'],
                low=data['low'],
                close=data['close'])])

# fig.layout = dict(xaxis=dict(type="category"))
fig.add_trace(go.Scatter(x=data.index, y= MA200, mode='lines', line=dict(width = 1),marker_color = 'red',
                        showlegend=True, name = 'MovingAverage200'))

fig.add_trace(go.Scatter(x=data.index, y= MA100, mode='lines', line=dict(width = 1),marker_color = 'green', 
                        showlegend=True, name = 'MovingAverage100'))

fig.add_trace(go.Scatter(x=data.index, y= MA14,mode='lines', line=dict(width = 1),marker_color = 'yellow',
                        showlegend=True, name = 'MovingAverage14'))

layout = go.Layout(
    plot_bgcolor='#efefef',
    # Font Families
    font_family='Monospace',
    font_color='#000000',
    font_size=15,
    xaxis=dict(
        rangeslider=dict(
            visible=False
        )
    )
)
fig.update_layout(layout)

fig.show()

In [3]:
def compare(Price,Red,Green,Yellow):

    # This function use built-in function sort to speed up the sort process.
    # Sort these 4 values (MA200,MA100,MA14,Close) in order to correctly label them from top to bottom

    #price:close price, Red = MovingAvg200, Green = MovingAvg100, Yellow = MovingAvg14
    price_list = {'Price':Price, 'Red':Red, 'Green':Green,'Yellow':Yellow}
    # sort the dictionary base on the dict.values
    sorted_price_list = (sorted(price_list.items(), key=lambda x: x[1],reverse =True))
    # eg sorted_price_list dict(("Price":196.820)  ('Red':196.820) ('Yellow':196.780) ('Green':196.750))
    label = ''
    for i in sorted_price_list:
        # use dict keys and the first capital letter for labelling
        label+=i[0][0]
    # print(sorted_price_list,label)
    return label

def distance(Price,Red,Green,Yellow):

    # This function calculate the distance (%) between 4 lines (MA200,MA100,MA14,Close)
    # use the sort function to sort them base on the dict.values  return (max, second, third, fourth)
    # then calculate the difference between them and divide by the range so the percentage values add up to 100%   

    #price:close price, Red = MovingAvg200, Green = MovingAvg100, Yellow = MovingAvg14
    price_list = {'Price':Price, 'Red':Red, 'Green':Green,'Yellow':Yellow}
    # sort the dictionary base on the dict.values
    sorted_price_list = (sorted(price_list.items(), key=lambda x: x[1],reverse =True))
    # distance between the maximum and minimum
    range = sorted_price_list[0][1] - sorted_price_list[3][1]
    # distance between the maximum and the second (%)
    ratio_top = (sorted_price_list[0][1] - sorted_price_list[1][1]) / range 
    # distance between the second and the third (%)
    ratio_mid = (sorted_price_list[1][1] - sorted_price_list[2][1]) / range
    # distance between the third and the fourth (%)
    ratio_bottom = (sorted_price_list[2][1] - sorted_price_list[3][1]) / range

    return ratio_top,ratio_mid,ratio_bottom



def MA_PRGY_Task(data):  
    index = 0
    # moving average 200
    MA200 = data['close'].rolling(window =200).mean()
    # moving average 100
    MA100 = data['close'].rolling(window =100).mean()
    # moving average 14
    MA14 = data['close'].rolling(window =14).mean()

    # add 4 columns to the dataframe
    data['event'] = ''
    data['ratio top'] = ''
    data['ratio mid'] = ''
    data['ratio bottom'] = ''

    label_list = []
    for i in tqdm(range(200,len(data),1)):
        Price = data.close[i]
        Red = MA200[i]
        Green = MA100[i]
        Yellow = MA14[i]
        # sort them in order return the label
        label = compare(Price,Red,Green,Yellow)

        # calculate the distance ratio
        ratio_top,ratio_mid,ratio_bottom = distance(Price,Red,Green,Yellow)
        data['ratio top'][i] = ratio_top
        data['ratio mid'][i] = ratio_mid
        data['ratio bottom'][i] = ratio_bottom
        
        # first label append to the list start at 200 
        if len(label_list) == 0:
            data['event'][200] = label
        label_list.append(label)

        if len(label_list) > 1:
            # label added if is different from previou one
            if label_list[-1] != label_list[-2] :
                data['event'][i] = label
             # label not be added if is same from previou one
            elif label_list[-1] == label_list[-2] :
                data['event'][i] = ''

    print(f'output excel file: GBP_USD_M15_2005_202206')
    data.to_excel(f'C:\\Users\\HFY\\Desktop\\capstone\\w9\\GBP_USD_M15_2005_202206_events.xlsx',index=False)
    print(f'Finish index: GBP_USD_M15_2005_202206')

In [4]:
MA_PRGY_Task(data)

100%|██████████| 440663/440663 [10:14:57<00:00, 11.94it/s]       


output excel file: GBP_USD_M15_2005_202206
Finish index: GBP_USD_M15_2005_202206


In [ ]:
# # price:close price, Red = MovingAvg200, Green = MovingAvg100, Yellow = MovingAvg14
# # compare(Price, Red, Green, Yellow)
# compare(196,195,194,193)

# # if all the parameters are equal (output base on the input order)
# compare(196,196,196,196)
# # output:[('Price', 2), ('Red', 2), ('Green', 2), ('Yellow', 2)]  PRGY

# # two of them are equal (same values return order also base on the input order)
# compare(1,2,2,1)
# # output: [('Red', 2), ('Green', 2), ('Price', 1), ('Yellow', 1)] RGPY

# #three of them are equal (same values return order also base on the input order)
# compare(2,1,2,2)
# # output: [('Price', 2), ('Green', 2), ('Yellow', 2), ('Red', 1)] PGYR

# # if the input values are the same, it will return the order base on the input sequence.

In [ ]:
# Test
data  = data[0:1000]
MA_PRGY_Task(data,0)
# output result
result = dict(data.event.value_counts())
for idx,values in result.items():
    if idx != '':
        print(f'{idx} {values} times')

RGYP 54 times
RGPY 50 times
PYGR 15 times
GRYP 13 times
YPGR 12 times
GRPY 12 times
GPRY 10 times
RPGY 10 times
PYRG 9 times
YGPR 7 times
GYRP 7 times
YPRG 6 times
YRPG 6 times
GYPR 5 times
RYGP 4 times
RPYG 3 times
PGYR 2 times
GPYR 2 times
YRGP 2 times
RYPG 2 times
YGRP 1 times
PRYG 1 times
PGRY 1 times


## Trading Strategy Part